In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import re
import collections
import pickle
import time
import random

In [3]:
URL = 'https://birdsoftheworld.org/bow/specieslist'
#URL = 'https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/specieslist'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

BOWlist = soup.find_all('a')

URL = 'https://birdsoftheworld.org/bow/specieslist'
#URL = 'https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/specieslist'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

'''
# Create a list with links
BOWpages = ['https://birdsoftheworld.org' + pages.get('href').replace('introduction', '') for pages in BOWlist 
                   if pages.get('href') != None
                   if pages.get('href').startswith('/bow/species')]

'''
# Create a list with links
BOWpages = ['https://birdsoftheworld-org.ezproxy.library.wur.nl' + pages.get('href').replace('introduction', '') for pages in BOWlist 
                   if pages.get('href') != None
                   if pages.get('href').startswith('/bow/species')]

In [3]:
# Init driver
browser = webdriver.Safari()

In [4]:
# Login URL
browser.get("https://edepot.wur.nl/541124")
# Load the website
time.sleep(8)

# Current page
main_page = browser.current_window_handle

# Hit Okay
Ok = browser.find_element_by_id("loginlinkBtn")
Ok.send_keys(u'\ue007')


time.sleep(3)

# changing the handles to access login page
for handle in browser.window_handles:
    if handle != main_page:
        login_page = handle
          
# change the control to signin page        
browser.switch_to.window(login_page)

# Username
username = browser.find_element_by_id("userNameInput")
username.clear()
username.send_keys("robert.vandevlasakker@wur.nl")

# Password
password = browser.find_element_by_name("Password")
password.clear()
password.send_keys("XXXXXXX")

# Hit Okay
Ok = browser.find_element_by_id("submitButton")
Ok.send_keys(u'\ue007')

time.sleep(8)

'''
# changing the handles to access login page
for handle in browser.window_handles:
    if handle != main_page or handle!= login_page:
        BOW = handle
'''

# change the control to signin page        
browser.switch_to.window(main_page)

time.sleep(5)

In [ ]:
'''
# Login URL
browser.get("https://secure.birds.cornell.edu/cassso/login?service=https%3A%2F%2Fbirdsoftheworld.org%2Flogin%2Fcas")

# Username
username = browser.find_element_by_id("input-user-name")
username.clear()
username.send_keys("RobertvdV")

# Password
password = browser.find_element_by_name("password")
password.clear()
password.send_keys("XXXXXXX")

# Sign in
SignIn = browser.find_element_by_id("form-submit")
SignIn.send_keys(u'\ue007')
'''

In [5]:
#####
# SHORTER ACCOUNTS SKIP?

# Init dict
DataBOW = collections.defaultdict(list)

# XML dump for testing
XMLdump = collections.defaultdict(list)

# Reference Removers
SourceRemover = [
    ' *\( *\d+\n\t.+?Close\n\t\n\)',
    ' *\( *\d+\n\t.+?Close\n\t\n\, .+Close\n\t\n\)',
    ' *\[\d+\n\t.+?Close\n\t\n\]',
    ' *\( *\d\d\n\t.+?Close\n\t\n,',
    ' *\(cf\..+?Close\n\t\n, .+Close\n\t\n\)',
    '\d+.+?Close\n\t\n'
]


#ReferenceRemover1 = ' *\( *\d+\n\t.+?Close\n\t\n\)'
#ReferenceRemover2 = ' *\( *\d+\n\t.+?Close\n\t\n\, .+Close\n\t\n\)'
# Loop over the test set
for Bird in BOWpages[0:500]:
    
    
    browser.get(Bird + 'appearance')
    #browser.get('https://birdsoftheworld.org/bow/species/norcar/cur/appearance')
    SleepTime = random.randint(2, 6)
    time.sleep(SleepTime)
    
    HTML = browser.page_source
    soup = BeautifulSoup(HTML, 'html.parser')

    # Bird name 
    if len(soup.title.text.split(' - ')) == 4:

        BirdName = soup.title\
                        .text\
                        .split(' - ')[1]\
                        .lstrip()
    else:
        BirdName = soup.title\
                    .text\
                    .split(' - ')[0]\
                    .lstrip()
        
    # Dump XML for testing
    XMLdump[BirdName].append(soup)
        
    # Find all text data
    Paragraphs = soup.find_all('p')

    # Loop over the text
    for Para in Paragraphs[1:]:
        try:

            # Check to which section the text belongs
            SubSection = Para.find_previous_siblings()[0]\
                             .text[0:50]\
                             .rstrip()\
                             .lstrip()
            #print(SubSection)

            # Check to which section the text belongs
            SubSubSection = Para.find_previous_siblings()[-1]\
                                 .text\
                                 .rstrip()\
                                 .lstrip()
            
            #print(SubSubSection)

        # If a text does not have a header, skip    
        except:
            continue
        
        
        ### THESE LINES MAY BE REMOVED###
        # Exceptions
        #if 'Formative' in SubSection:
        #    continue
        #if 'Juvenile' in SubSection:
        #    continue
        #if 'Immature' in SubSection:
        #    continue
        ### THESE LINES MAY BE REMOVED###
        
        # CHeck if Subsection is about juveniles
        if all(s not in SubSection for s in ['Immature', 'Juvenile', 'Formative', 'Chick', 'Natal']) == False:
            continue
        
        # Check if the information given in a text contains info about juveniles
        try:
            if Para.find('b').text.startswith(('Immature', 'Juvenile', 'Formative', 'Chick')) or\
            Para['class'][0] == 'u-text-2':
                    continue
        
        except:
            # See which are needed and append with 1
            if SubSection in ['Identification', 'Adult'] or\
               SubSubSection in ['Plumages', 'Bare Parts', 'Identification']:
        
                # Clean the text from references 
                TextCleaned = Para.text

                # Remove sources
                for Remover in SourceRemover:
                    TextCleaned = re.sub(Remover, '', TextCleaned, flags=re.DOTALL)

                # Drop useless short nonsense
                if len(TextCleaned.split()) <2:
                    continue
                else:
                    DataBOW[BirdName].append(tuple([1, TextCleaned]))

            # Otherwise label 2        
            else:

                # Clean the text from references 
                TextCleaned = Para.text

                # Remove sources
                for Remover in SourceRemover:
                    TextCleaned = re.sub(Remover, '', TextCleaned, flags=re.DOTALL)

                # Drop useless short nonsense
                if len(TextCleaned.split()) <2:
                    continue
                else:
                    DataBOW[BirdName].append(tuple([0, TextCleaned]))

In [6]:
with open('dataBOW.pkl', 'wb') as f:
    pickle.dump(DataBOW, f)

# With Measurements

In [13]:
#####
# SHORTER ACCOUNTS SKIP?

# Init dict
DataBOW = collections.defaultdict(list)

# XML dump for testing
XMLdump = collections.defaultdict(list)

# Reference Removers
SourceRemover = [
    ' *\( *\d+\n\t.+?Close\n\t\n\)',
    ' *\( *\d+\n\t.+?Close\n\t\n\, .+Close\n\t\n\)',
    ' *\[\d+\n\t.+?Close\n\t\n\]',
    ' *\( *\d\d\n\t.+?Close\n\t\n,',
    ' *\(cf\..+?Close\n\t\n, .+Close\n\t\n\)',
    '\d+.+?Close\n\t\n'
]


#ReferenceRemover1 = ' *\( *\d+\n\t.+?Close\n\t\n\)'
#ReferenceRemover2 = ' *\( *\d+\n\t.+?Close\n\t\n\, .+Close\n\t\n\)'
# Loop over the test set
for Bird in BOWpages[0:500]:
    
    
    browser.get(Bird + 'appearance')
    #browser.get('https://birdsoftheworld.org/bow/species/norcar/cur/appearance')
    SleepTime = random.randint(2, 6)
    time.sleep(SleepTime)
    
    HTML = browser.page_source
    soup = BeautifulSoup(HTML, 'html.parser')

    # Bird name 
    if len(soup.title.text.split(' - ')) == 4:

        BirdName = soup.title\
                        .text\
                        .split(' - ')[1]\
                        .lstrip()
    else:
        BirdName = soup.title\
                    .text\
                    .split(' - ')[0]\
                    .lstrip()
        
    # Dump XML for testing
    XMLdump[BirdName].append(soup)
        
    # Find all text data
    Paragraphs = soup.find_all('p')

    # Loop over the text
    for Para in Paragraphs[1:]:
        try:

            # Check to which section the text belongs
            SubSection = Para.find_previous_siblings()[0]\
                             .text[0:50]\
                             .rstrip()\
                             .lstrip()
            #print(SubSection)

            # Check to which section the text belongs
            SubSubSection = Para.find_previous_siblings()[-1]\
                                 .text\
                                 .rstrip()\
                                 .lstrip()
            
            #print(SubSubSection)

        # If a text does not have a header, skip    
        except:
            continue
        
        
        ### THESE LINES MAY BE REMOVED###
        # Exceptions
        #if 'Formative' in SubSection:
        #    continue
        #if 'Juvenile' in SubSection:
        #    continue
        #if 'Immature' in SubSection:
        #    continue
        ### THESE LINES MAY BE REMOVED###
        
        # CHeck if Subsection is about juveniles
        if all(s not in SubSection for s in ['Immature', 'Juvenile', 'Formative', 'Chick', 'Natal']) == False:
            continue
        
        # Check if the information given in a text contains info about juveniles
        try:
            if Para.find('b').text.startswith(('Immature', 'Juvenile', 'Formative', 'Chick')) or\
            Para['class'][0] == 'u-text-2':
                    continue
        
        except:
            # See which are needed and append with 1
            if SubSection in ['Identification', 'Adult', 'Measurements'] or\
               SubSubSection in ['Plumages', 'Bare Parts', 'Identification', 'Measurements']:
        
                # Clean the text from references 
                TextCleaned = Para.text

                # Remove sources
                for Remover in SourceRemover:
                    TextCleaned = re.sub(Remover, '', TextCleaned, flags=re.DOTALL)

                # Drop useless short nonsense
                if len(TextCleaned.split()) <2:
                    continue
                else:
                    DataBOW[BirdName].append(tuple([1, TextCleaned]))

            # Otherwise label 2        
            else:

                # Clean the text from references 
                TextCleaned = Para.text

                # Remove sources
                for Remover in SourceRemover:
                    TextCleaned = re.sub(Remover, '', TextCleaned, flags=re.DOTALL)

                # Drop useless short nonsense
                if len(TextCleaned.split()) <2:
                    continue
                else:
                    DataBOW[BirdName].append(tuple([0, TextCleaned]))

In [14]:
with open('dataBOW_withMeasurements.pkl', 'wb') as f:
    pickle.dump(DataBOW, f)

# All Pages

In [16]:
# Init dict
DataBOW = collections.defaultdict(list)

# XML dump for testing
XMLdump = collections.defaultdict(list)

# Loop over the test set
for Bird in BOWpages[0:10]:
    
    browser.get(Bird + 'appearance')
    #browser.get('https://birdsoftheworld.org/bow/species/norcar/cur/appearance')
    SleepTime = random.randint(2, 6)
    time.sleep(SleepTime)
    
    HTML = browser.page_source
    soup = BeautifulSoup(HTML, 'html.parser')

    # Bird name 
    BirdName = soup.title.text.split(' - ')[1].lstrip()
    
    # Dump XML for testing
    XMLdump[BirdName].append(soup)
        
    # Find all text data
    Paragraphs = soup.find_all('p')

    # Loop over the text
    for Para in Paragraphs[1:]:
        if Para.startswith("Editor's Note: This is a shorter format account"):
            continue

In [1]:
DataBOW.keys()

NameError: name 'DataBOW' is not defined

In [22]:
soup.title\
                        .text\
                        .split(' - ')[1]\
                        .lstrip()

'Highland Tinamou'